# 1) 손가락으로 드래그 앤 드롭

### https://google.github.io/mediapipe/solutions/hands - 미디어 파이프
### https://www.youtube.com/watch?v=6DxN8G9vB50 - 유튜브

In [1]:
import cv2
import mediapipe as mp
import numpy as np 
import math 

# 드로잉 툴 
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# For webcam input:
cap = cv2.VideoCapture(0) # 기본캠 설정
cap.set (3, 1280) # 윈도우 창 가로길이
cap.set (4, 720) # 윈도우 창 세로길이
colorR = (125, 255, 50) # 사각형 기본컬러 설정
colorC = (255,0,0) # 사각형 코너 라인 기본컬러 설정

cx, cy, r_w, r_h = 100,100,200,200 # 동적 변수로 들어가기 위해 사각형 치수를 따로 변수에 담음


# 클래스 생성 ( 사각형 만드는 클래스 생성 )
class DragRect(): 
  # 메소드 생성 ( 초기화를 위한 메소드 )
  def __init__(self, posCenter, size = [200,200]): # 사각형의 좌표를 지정 ( posCenter = cx, xy / size = r_w, r_h )
    self.posCenter = posCenter # 사각형의 행,열
    self.size = size # 사각형의 높이,너비

  # 메소드 생성 ( 검지 끝 좌표에 사각형을 업데이트 해주는 메소드 )
  def update(self, cursor):
      cx, cy = self.posCenter # 밑에 if 문이 작동하게 하기 위해 작성 ( 사각형의 정보는 윗 클래스에서 가져옴 ) 
      r_w, r_h = self.size

      # 만약 검지손가락 끝이 사각형 영역에 있다면 self.posCenter 를 cursor의 좌표로 변경 ?
      # ( 사각형 내부에 내 검지손가락 좌표가 있는지 확인하는걸 가져다 쓰기위해 아래에서 썼던 if문을 가져옴 )
      if cx-r_w//2 < cursor[0] < cx+r_w//2 and cy-r_h//2 < cursor[1] < cy+r_h//2:
        self.posCenter = cursor # 사각형 행,열의 중심이 검지 끝 좌표에 업데이트 됨

rectList = [] # 빈 리스트 생성
for x in range(10): # 사각형 5개를 그리기 위해 range로 범위 설정.
  rectList.append(DragRect([x*250+150,150])) # x에 1~5가 들어와서 x값이 바뀌어서 사각형이 옆으로 나란히 표시됨 
                                             # 이 생성된 사각형들을 위에 만든 빈 리스트에 어펜드
# 코너 사각형 함수 추가
def cornerRect(img, bbox, l=30, t=5, rt=1,
               colorR=(255, 0, 255), colorC=(0, 0, 255)):
    """
    :param img: Image to draw on.
    :param bbox: Bounding box [x, y, w, h]
    :param l: length of the corner line
    :param t: thickness of the corner line
    :param rt: thickness of the rectangle
    :param colorR: Color of the Rectangle
    :param colorC: Color of the Corners
    :return:
    """
    x, y, w, h = bbox
    x1, y1 = x + w, y + h
    if rt != 0:
        cv2.rectangle(img, bbox, colorR, rt)
    # Top Left  x,y
    cv2.line(img, (x, y), (x + l, y), colorC, t)
    cv2.line(img, (x, y), (x, y + l), colorC, t)
    # Top Right  x1,y
    cv2.line(img, (x1, y), (x1 - l, y), colorC, t)
    cv2.line(img, (x1, y), (x1, y + l), colorC, t)
    # Bottom Left  x,y1
    cv2.line(img, (x, y1), (x + l, y1), colorC, t)
    cv2.line(img, (x, y1), (x, y1 - l), colorC, t)
    # Bottom Right  x1,y1
    cv2.line(img, (x1, y1), (x1 - l, y1), colorC, t)
    cv2.line(img, (x1, y1), (x1, y1 - l), colorC, t)

    return img


# 웹캠, 영상 파일의 경우 이것을 사용하세요.
with mp_hands.Hands(
  model_complexity = 0, # 모델 복잡성 기본값 = 1
  max_num_hands = 1, # 손 하나만 잡아주는 옵션 추가 디폴트가 ( 2 )
  min_detection_confidence = 0.5, # 최소 탐지 신뢰도 기본값 = 0.5
  min_tracking_confidence = 0.5,) as hands: # 최소 추적 신뢰도 기본값 = 0.5
  while cap.isOpened():
    success, image = cap.read() # 프레임과 이미지 표시
    if not success:
      print("Ignoring empty camera frame.")
      # 동영상을 불러올 경우는 'continue' 대신 'break'를 사용합니다.
      break

    # 필요에 따라 성능 향상을 위해 이미지 작성을 불가능함으로 기본 설정합니다.
    image.flags.writeable = False 
    image = cv2.flip(image, 1) # 보기 편하기 위해 이미지를 좌우를 반전후 실행 ( 1 = 수평 0 = 수직 )
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # 이미지에 손 주석을 그립니다.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        # 기본적으로 미디어파이프 사이트에 있는 손 좌표 그려주는 코드
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

        # 검지 끝 엄지 끝 좌표잡기
        index_finger_tip_x = hand_landmarks.landmark[8].x # 검지손가락 끝 포인트 좌표
        index_finger_tip_y = hand_landmarks.landmark[8].y
        middle_finger_tip_x = hand_landmarks.landmark[12].x # 중지손가락 끝 포인트 좌표
        middle_finger_tip_y = hand_landmarks.landmark[12].y

        # 이미지 전체 크기를 바탕으로 실제 좌표로 반환함 ( 픽셀값 )
        h, w, _ = image.shape 
        index_finger_tip = (int(index_finger_tip_x*w), int(index_finger_tip_y*h))
        middle_finger_tip = (int(middle_finger_tip_x*w), int(middle_finger_tip_y*h))

        # 손가락 사이 거리 구하기 ( 픽셀값 )
        distance = math.dist((index_finger_tip[0],index_finger_tip[1]), (middle_finger_tip[0], middle_finger_tip[1]))
        # print(distance) # 검지 엄지 사이 거리 표시
        cv2.putText(image, f'{int(distance)} px', # 손가락 사이의 거리를 화면상에 텍스트로 표시 ( int로 표시해 소수점을 없앰 ) 
        (middle_finger_tip[0]-30,middle_finger_tip[1]-75), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

        # 40픽셀 이하로 거리좁히면 클릭 프린트
        if distance < 40:
          cursor = index_finger_tip # 검지 손가락 끝 랜드마크
          cv2.putText(image, "click !" , (index_finger_tip[0], index_finger_tip[1]-50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
          # 업데이트 메서드 불러오기 ( 업데이트 메서드에 포함시킨 if문 )
          for rect in rectList:   # 사각형을 하나씩 꺼냄
            rect.update(cursor)   # 손가락이 어떤 사각형을 가리키는지 확인하는 작업
        
        # 좌표로 잡은 검지 끝, 중지 끝 잡아서 원 그리기
        cv2.circle(image, index_finger_tip, 20, (0,0,0), -1, cv2.LINE_AA)
        cv2.circle(image, middle_finger_tip, 20, (0,0,0), -1, cv2.LINE_AA)
        # cv2.circle(image, , 20, (0,0,0), 2, cv2.LINE_AA)
        # 손가락 사이 거리에 서로 이어주는 선 긋기
        cv2.line(image, index_finger_tip, middle_finger_tip, (0,0,0), 2, 8)
  

    ### 투명도 사각형 그리기 ###
    imgNew = np.zeros_like(image, np.uint8) # image의 shape만큼 0으로 채움(tuple,int형태)
    for rect in rectList:
      # 지정한 클래스로 사각형 그리기
      cx, cy = rect.posCenter # 사각형의 행,열
      r_w, r_h = rect.size # 사각형의 높이,너비
      # 드래그 앤 드롭할때 쓸 사각형 그리기
      cv2.rectangle(imgNew, (cx-r_w//2,cy-r_h//2), (cx+r_w//2,cy+r_h//2), colorR, cv2.FILLED)
      cornerRect (imgNew, (cx-r_w//2, cy-r_h//2, r_w, r_h), 20, rt = 0) # 코너에 라인추가 하는 함수 불러오기

    out = image.copy() # 원본의 복사본 생성해서 담기
    alpha = 0.25 # 투명도 설정 0 = 완전 불투명, 1 = 완전 투명
    mask = imgNew.astype(bool) # 참, 거짓 자료형으로 변환
    out[mask] = cv2.addWeighted(image, alpha, imgNew, 1 - alpha, 0) [mask]
    
    # 보기 편하게 이미지를 좌우 반전합니다.
    # cv2.imshow('MediaPipe Hands', cv2.flip(image, 1)) # << 원래 이미지 ( 과정 알아보기 쉽게 주석처리함 )
    cv2.imshow('MediaPipe Hands', out) # 투명도 박스로 교체하기 위해 원래 이미지에서 out을 출력하는걸로 바꿈
    if cv2.waitKey(5) & 0xFF == 27:
      break

cap.release() # 지정 객체 해제
cv2.destroyAllWindows() # 모든 윈도우 창 제거